# LABELS

In [1]:
import os
import re

In [4]:
dataSetMLF_FilesPath=r'E:\UAspeesh dataset\mlf'
dataSetMLF_FilesName=[file for file in os.listdir(dataSetMLF_FilesPath)if file.endswith('.mlf')]

In [5]:
dataSetTxtFilesPath=r'C:\Users\Ahmed Salah\Desktop\files'
dataSetTxtFilesName=[file for file in os.listdir(dataSetTxtFilesPath)if file.endswith('.txt')]

In [7]:
dataSetMLF_FilesName

['CF02_word.mlf',
 'CF03_word.mlf',
 'CF04_word.mlf',
 'CM04_word.mlf',
 'CM05_word.mlf',
 'CM06_word.mlf',
 'CM08_word.mlf',
 'CM09_word.mlf',
 'CM10_word.mlf',
 'CM12_word.mlf',
 'CM13_word.mlf',
 'F02_word.mlf',
 'F03_word.mlf',
 'F04_word.mlf',
 'M01_word.mlf',
 'M04_word.mlf',
 'M05_word.mlf',
 'M06_word.mlf',
 'M07_word.mlf',
 'M08_word.mlf',
 'M09_word.mlf',
 'M10_word.mlf',
 'M11_word.mlf',
 'M12_word.mlf',
 'M13_word.mlf',
 'M14_word.mlf',
 'M15_word.mlf',
 'M16_word.mlf']

In [8]:
dataSetTxtFilesName

['labels.txt']

## Convert MLF Files To one Dict {FileName:Word}

In [3]:
def convertFromMlfToText(mlf_files,dataSetMLF_FilesPath):
    allmlf_FilesInDict,errorReadMlfFiles={},[]
    for mlf_File in mlf_files:
        try: 
            with open(dataSetMLF_FilesPath+'\\'+mlf_File,mode='r',encoding='utf-8') as mlf_FileBuffer:
                mlf_FileBufferWords=mlf_FileBuffer.readlines()
            CounterOfmlf_FileBufferWords=0
            for line in mlf_FileBufferWords:
                CounterOfmlf_FileBufferWords+=1
                if re.match(r'"\*/\w+.lab"',line):
                    allmlf_FilesInDict[re.sub('.lab','.wav',re.sub('"\n','',re.sub(r'"\*/','',line)))]=mlf_FileBufferWords[CounterOfmlf_FileBufferWords]
        except:
            errorReadMlfFiles.append(mlf_File)

    return allmlf_FilesInDict,errorReadMlfFiles

In [4]:
%%time
allmlf_FilesInDict,errorReadMlfFiles=convertFromMlfToText(UASpeechMLF_FilesName,UASpeechMLF_FilesPath)

Wall time: 2.76 s


In [5]:
print(len(allmlf_FilesInDict))
print(len(errorReadMlfFiles))

107100
0


## convert all mlf files to one txt file as Dict (file:word)

In [37]:
def convertDictToFile(Dict,nameOfFile): 
    if not os.path.isdir('files'):
        os.mkdir('files')
    with open('files/'+nameOfFile+'.txt','w') as newLabelFile:
            for key in Dict:
                newLabelFile.writelines(key+':'+Dict[key].strip()+'\n')

In [9]:
convertDictToFile(allmlf_FilesInDict,'labels')

## Read From Text file {FileName:Word}

In [9]:
def readLabelsFromTextFileAsDict(labelsFiles,DataSetPath):
    allLabelsFilesAsDict,errorReadLabelFiles={},[]
    for File in labelsFiles:
        try:
            with open(DataSetPath+'\\'+File,mode='r') as labelsFile:
                for line in sorted(labelsFile.read().split('\n')):
                    line=line.split(':')
                    allLabelsFilesAsDict[line[0]]=line[-1]
        except:
            errorReadLabelFiles.append(File)
    return allLabelsFilesAsDict,errorReadLabelFiles
    

In [15]:
%%time
allLabelsFilesAsDict,errorReadLabelFiles=readLabelsFromTextFileAsDict(dataSetTxtFilesName,dataSetTxtFilesPath)

Wall time: 99.7 ms


In [16]:
print(len(allLabelsFilesAsDict))
print(len(errorReadLabelFiles))

107101
0


## Convert MLF Files To Multi TXT Files (107100) in Folder (files)

In [2]:
def convertFromMlfToTxtFiles(dataSetMLF_Files,dataSetMLF_FilesPath):
    if not os.path.isdir('files'):
        os.mkdir('files')
    ErrorFileNameInLineDict={}
    for mlf_File in dataSetMLF_Files: 
        with open(dataSetMLF_FilesPath+'\\'+mlf_File,mode='r',encoding='utf-8') as mlf_FileBuffer:
            mlf_FileBufferWords=mlf_FileBuffer.readlines()
        CounterOfmlf_FileBufferWords=0
        for line in mlf_FileBufferWords:
            CounterOfmlf_FileBufferWords+=1
            if re.match(r'"\*/\w+.lab"',line):
                try:
                    with open('files/'+re.sub('.lab"\n','',re.sub(r'"\*/','',line))+'.txt','w') as WordFile:
                        WordFile.write(mlf_FileBufferWords[CounterOfmlf_FileBufferWords])
                except:
                    ErrorFileNameInLineDict[mlf_File+line]=CounterOfmlf_FileBufferWords
    return ErrorFileNameInLineDict


## Print If There Erorr OR Not 

In [ ]:
print(ErrorFileNameInLineDict)

# AUDIO

In [18]:
import os
dataSetMainFilesOfDataPath=r'G:\UASpeech\audio' #edit path
dataSetMainFilesOfDataName=[folder for folder in os.listdir(dataSetMainFilesOfDataPath)if folder.endswith('_audio')] #to get only Folders

In [20]:
print(dataSetMainFilesOfDataName) # all Main Files in dataset

['CF02_audio', 'CF03_audio', 'CF04_audio', 'CM04_audio', 'CM05_audio', 'CM08_audio', 'CM09_audio', 'CM10_audio', 'CM12_audio', 'CM13_audio', 'F02_audio', 'F03_audio', 'F04_audio', 'M01_audio', 'M04_audio', 'M05_audio', 'M07_audio', 'M08_audio', 'M09_audio', 'M10_audio', 'M11_audio', 'M12_audio', 'M14_audio', 'M16_audio']


## get All Audio Files Paths

In [21]:
def getAudioFilesPath(DatasetPath,dataSetMainFiles):
    dataSetAudioFilesPaths=[]
    for mainFile in dataSetMainFiles:#main Files as CF02
        for mainFileChiled in sorted(os.listdir(DatasetPath+'\\'+mainFile)):#git records in every main file 
            if mainFileChiled.endswith('.wav'):
                dataSetAudioFilesPaths.append(DatasetPath+'\\'+mainFile+'\\'+mainFileChiled)#append Path of file .wav
    return dataSetAudioFilesPaths


In [22]:
%%time
dataSetAudioFilesPaths=getAudioFilesPath(dataSetMainFilesOfDataPath,dataSetMainFilesOfDataName)

Wall time: 132 ms


In [23]:
print(len(dataSetAudioFilesPaths))
print(dataSetAudioFilesPaths[0])
print(dataSetAudioFilesPaths[-1])


90460
G:\UASpeech\audio\CF02_audio\CF02_B1_C10_M3.wav
G:\UASpeech\audio\M16_audio\M16_B3_UW9_M8.wav


## Delete All Audio Files That Haven't Label

In [30]:
def deleteAudioFilesWithoutLabel(dataSetAudioFilesPaths,allLabelsFilesAsDict):
    audioFilesHadDeleted=[]
    dataSetAudioFilesPathsCopy=dataSetAudioFilesPaths.copy()
    ErrorRemovedFile=[]
    for audioFilePath in dataSetAudioFilesPaths:
        if audioFileName not in allLabelsFilesAsDict:
            try:
                audioFilesHadDeleted.append(audioFilePath)
                os.remove(audioFilePath)
                dataSetAudioFilesPathsCopy.remove(audioFilePath)
            except:
                ErrorRemovedFile.append(audioFilePath)
    return dataSetAudioFilesPathsCopy,audioFilesHadDeleted,ErrorRemovedFile

In [25]:
%%time
dataSetAudioFilesPaths,audioFilesHadDeleted,ErrorRemovedFile=deleteAudioFilesWithoutLabel(dataSetAudioFilesPaths,allLabelsFilesAsDict)

In [28]:
print(len(dataSetAudioFilesPaths)) #all paths after update
print(len(audioFilesHadDeleted)) #if there file had remove
print(len(ErrorRemovedFile)) #if there file have error and didn't remove

90460
0


## Delete All labels That Haven't Audio File

In [33]:
def deleteLabelWithoutAudio(allLabelsFilesAsDict,dataSetAudioFilesPaths):
    labelLinesHadDeleted=[]
    dataSetAudioFilesNames=[audioFileName.split('\\')[-1] for audioFileName in dataSetAudioFilesPaths ]
    allLabelsFilesAsDictCopy=allLabelsFilesAsDict.copy()
    for mlf_FileWord in allLabelsFilesAsDict:
        if mlf_FileWord not in dataSetAudioFilesNames:
            labelLinesHadDeleted.append(mlf_FileWord)
            allLabelsFilesAsDictCopy.pop(mlf_FileWord)
    del(dataSetAudioFilesNames)
    return allLabelsFilesAsDictCopy,labelLinesHadDeleted

In [34]:
%%time
allLabelsFilesAsDict,labelLinesHadDeleted=deleteLabelWithoutAudio(allLabelsFilesAsDict,dataSetAudioFilesPaths)

Wall time: 1min 20s


In [35]:
print(len(allLabelsFilesAsDict)) # labels after update
print(len(labelLinesHadDeleted)) #lines that deleted

90460
16641


In [39]:
convertDictToFile(allLabelsFilesAsDict,'labels') #convert labels Dict to file after update

# Rename 

In [40]:
import re
def renameAllFiles(DatasetPath,dataSetMainFiles,LabelsFilesAsDict):
    newLabelsDict,formOldeToNew,errorRenameFile={},{},[]
    for mainFile in dataSetMainFiles:#main Files as CF02
        audioNumber=0
        for mainFileChiled in os.listdir(DatasetPath+'\\'+mainFile):#git records in every main file 
            if mainFileChiled.endswith('.wav'):
                audioNumber+=1
                newText=re.sub('_audio','_',mainFile)
                for zeros in range((4-len(str(audioNumber)))):
                    newText+='0'
                newFileName=newText+ str(audioNumber) + '.wav'
                oldFilePath=( DatasetPath+'\\'+mainFile+'\\'+mainFileChiled)
                newFilePath=( DatasetPath+'\\'+mainFile+'\\'+newFileName)
                newLabelsDict[newFileName]=LabelsFilesAsDict[mainFileChiled]
                formOldeToNew[mainFileChiled]=newFileName
                try:
                    os.rename(oldFilePath,newFilePath)
                    print(mainFileChiled)
                except:
                    errorRenameFile.append(mainFileChiled)
    return newLabelsDict,formOldeToNew,errorRenameFile
                

In [41]:
%%time
allLabelsFilesAsDict,formOldeToNew,errorRenameFile=renameAllFiles(dataSetMainFilesOfDataPath,dataSetMainFilesOfDataName,allLabelsFilesAsDict)

_B3_UW67_M7.wav
M07_B3_UW67_M8.wav
M07_B3_UW68_M3.wav
M07_B3_UW68_M5.wav
M07_B3_UW68_M6.wav
M07_B3_UW68_M7.wav
M07_B3_UW68_M8.wav
M07_B3_UW69_M3.wav
M07_B3_UW69_M5.wav
M07_B3_UW69_M6.wav
M07_B3_UW69_M7.wav
M07_B3_UW69_M8.wav
M07_B3_UW6_M3.wav
M07_B3_UW6_M5.wav
M07_B3_UW6_M6.wav
M07_B3_UW6_M7.wav
M07_B3_UW6_M8.wav
M07_B3_UW70_M3.wav
M07_B3_UW70_M5.wav
M07_B3_UW70_M6.wav
M07_B3_UW70_M7.wav
M07_B3_UW70_M8.wav
M07_B3_UW71_M3.wav
M07_B3_UW71_M5.wav
M07_B3_UW71_M6.wav
M07_B3_UW71_M7.wav
M07_B3_UW71_M8.wav
M07_B3_UW72_M3.wav
M07_B3_UW72_M5.wav
M07_B3_UW72_M6.wav
M07_B3_UW72_M7.wav
M07_B3_UW72_M8.wav
M07_B3_UW73_M3.wav
M07_B3_UW73_M5.wav
M07_B3_UW73_M6.wav
M07_B3_UW73_M7.wav
M07_B3_UW73_M8.wav
M07_B3_UW74_M3.wav
M07_B3_UW74_M5.wav
M07_B3_UW74_M6.wav
M07_B3_UW74_M7.wav
M07_B3_UW74_M8.wav
M07_B3_UW75_M3.wav
M07_B3_UW75_M5.wav
M07_B3_UW75_M6.wav
M07_B3_UW75_M7.wav
M07_B3_UW75_M8.wav
M07_B3_UW76_M3.wav
M07_B3_UW76_M5.wav
M07_B3_UW76_M6.wav
M07_B3_UW76_M7.wav
M07_B3_UW76_M8.wav
M07_B3_UW77_M3.wav
M

In [42]:
print(len(errorRenameFile))


0


In [44]:
convertDictToFile(allLabelsFilesAsDict,'labels')
convertDictToFile(formOldeToNew,'formOldeToNew')
